In [2]:
library(tidyverse)
library(ggplot2)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [56]:
twitch_data <- read_csv("https://raw.githubusercontent.com/BraedenFong12/DSCI-100-Group-69/main/Dataset")
colnames(twitch_data) <- make.names(colnames(twitch_data), unique = TRUE)

twitch_data <- twitch_data |>
select(Channel, Mature, Partnered, Followers, Average.viewers)
twitch_data

Rows: 1000 Columns: 11
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Channel, Language
dbl (7): Watch time(Minutes), Stream time(minutes), Peak viewers, Average vi...
lgl (2): Partnered, Mature

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Channel,Mature,Partnered,Followers,Average.viewers
<chr>,<lgl>,<lgl>,<dbl>,<dbl>
xQcOW,FALSE,TRUE,3246298,27716
summit1g,FALSE,TRUE,5310163,25610
Gaules,TRUE,TRUE,1767635,10976
ESL_CSGO,FALSE,TRUE,3944850,7714
Tfue,FALSE,TRUE,8938903,29602
Asmongold,FALSE,TRUE,1563438,42414
NICKMERCS,FALSE,TRUE,4074287,24181
Fextralife,FALSE,TRUE,508816,18985
loltyler1,FALSE,TRUE,3530767,22381


In [18]:
dt = sort(sample(nrow(twitch_data), nrow(twitch_data)*.7))
training_data <- twitch_data[dt,]
testing_data <- twitch_data[-dt,]

In [69]:
observations_mature <- training_data |> 
group_by(Mature) |>
tally()

observations_partnered <- training_data |> 
group_by(Partnered) |>
tally()

observations_followers <- training_data |> 
map(mean)

observations_na <- training_data |> 
is.na() |>
colSums(is.na(training_data))

twitch_table <- table(observations_mature, observations_partnered, observations_followers, observations_na)

Warning message in xtfrm.data.frame(x):
“cannot xtfrm data frames”
Warning message in xtfrm.data.frame(x):
“cannot xtfrm data frames”


ERROR: Error in table(observations_mature, observations_partnered, observations_followers, : all arguments must have the same length
